In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


leagueId, seasonId = 1950807, 2018
slot = 0

df = pd.DataFrame(columns=['Player', 'Owner', 'Pts', 'Week'])

print('Week ', end='')
for week in range(1, 2):
    print('%d .. ' % week, end='')
    
    # do first two "pages" of results
    for si in [0, 50]:
        r = requests.get('http://games.espn.com/ffl/leaders',
                         params={'leagueId': leagueId, 'seasonId': seasonId, 
                                 'scoringPeriodId': week,
                                 #'slotCategoryId': slot,
                                 'startIndex': si})
                         #,cookies={'SWID': swid, 'espn_s2': espn})
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table', class_='playerTableTable')
        tdf = pd.read_html(str(table), flavor='bs4')[0]  # returns a list of df's, grab first
        tdf = tdf.iloc[2:, [0,2,26]].reset_index(drop=True)
        tdf.columns = ['Player', 'Owner', 'Pts']
        #tdf['Pts'] = tdf['Pts'].fillna(0).astype('float')
        tdf['Player'] = tdf['Player'].str.split(',').str[0]  # keep just player name
        tdf['Week'] = week

        df = df.append(tdf)
        
            # do first 3-4 "pages" of results
    for si in [100, 150]:
        r = requests.get('http://games.espn.com/ffl/leaders',
                         params={'leagueId': leagueId, 'seasonId': seasonId, 
                                 'scoringPeriodId': week,
                                 #'slotCategoryId': slot,
                                 'startIndex': si})
                         #,cookies={'SWID': swid, 'espn_s2': espn})
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table', class_='playerTableTable')
        tdf = pd.read_html(str(table), flavor='bs4')[0]  # returns a list of df's, grab first
        tdf = tdf.iloc[2:, [0,2,26]].reset_index(drop=True)
        tdf.columns = ['Player', 'Owner', 'Pts']
        tdf['Pts'] = tdf['Pts'].fillna(0)#.astype('float')
        tdf['Player'] = tdf['Player'].str.split(',').str[0]  # keep just player name
        tdf['Week'] = week

        df = df.append(tdf)
        

        
       
print('Complete.')
#print(df)
df.to_csv('df test.csv')


Week 1 .. Complete.


In [2]:
print(df.loc[df['Player'] == 'Julio Jones'])

         Player Owner Pts Week
17  Julio Jones  EAME  23    1


In [4]:

df_int = pd.read_csv('Fantasy Example - Sheet1.csv', skip_blank_lines=False)
print(df_int)

list_names = []
for x in df_int['Team Name']:
    if x not in list_names:
        list_names.append(x)
df_names = pd.DataFrame(list_names, columns=['Player Names'])

df_filter = pd.merge(df, df_names, how='right', left_on='Player', right_on='Player Names')

print(df_filter)


    
  

    

             Team Name  Example Team
0   Ben Roethlisberger           NaN
1         Alvin Kamara           NaN
2         Le'Veon Bell           NaN
3      Jamaal Williams           NaN
4          Tyreek Hill           NaN
5        Blake Bortles           NaN
6          Josh Gordon           NaN
7                  NaN           NaN
8         Tyrod Taylor           NaN
9          Kareem Hunt           NaN
10       Melvin Gordon           NaN
11        Adam Thielen           NaN
12         Carlos Hyde           NaN
13     Deandre Hopkins           NaN
14      Jameis Winston           NaN
                Player     Owner   Pts Week        Player Names
0          Tyreek Hill        KK  38.8    1         Tyreek Hill
1         Alvin Kamara      CUDD  38.6    1        Alvin Kamara
2         Tyrod Taylor  WA (Wed)  23.6    1        Tyrod Taylor
3        Melvin Gordon        KK  23.1    1       Melvin Gordon
4        Blake Bortles  WA (Wed)  13.2    1       Blake Bortles
5         Adam Thielen  

In [5]:
#s = (df_int.loc[df_int['Team Name'] == 'Josh Gordon'])
#print(df_int['Team Name'].iloc[1])
for x in range(len(df_int)):
    name = df_int['Team Name'].iloc[x]
    s = (df_filter.loc[df_filter['Player'] == name])
    #print(s)
    values = (df_filter.loc[df_filter['Player'] == name])['Pts']
    points = np.sum(values)
    df_int['Example Team'].iloc[x] = float(points)
    
#df_int.loc['Total'] = df_int['Example Team'].sum()
df_int['Team Name'].iloc[7] = 'Total 1'
df_int['Example Team'].iloc[7] = df_int['Example Team'][0:7].sum()
df_int.loc[15] = ['Total 2',df_int['Example Team'][8:15].sum()] 

print(df_int)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


             Team Name  Example Team
0   Ben Roethlisberger           9.0
1         Alvin Kamara          38.6
2         Le'Veon Bell           0.0
3      Jamaal Williams           4.7
4          Tyreek Hill          38.8
5        Blake Bortles          13.2
6          Josh Gordon           8.2
7              Total 1         112.5
8         Tyrod Taylor          23.6
9          Kareem Hunt           4.9
10       Melvin Gordon          23.1
11        Adam Thielen          13.2
12         Carlos Hyde          13.0
13     Deandre Hopkins           0.0
14      Jameis Winston           0.0
15             Total 2          77.8
